In [4]:
import sys
sys.path.append(r'./script/')
import copy
import numpy as np
import plotly.graph_objects as go
from ECMpy_function import *
from cobra.flux_analysis import flux_variability_analysis

# Input and output files

In [5]:
json_model_file='./model/ecMTM.json'
sbml_path = "./model/iYT1475.xml"
ecGEM_rxn_file ='./data/ecGEM_reactions_list.csv'
GEM_rxn_file = './data/GEM_reactions_list.csv'

obj='Biomass'
substrate='R2399'
umax=0.306 
def_sub_cons=-10

fva_ec773max_outfile='./analysis/fva_ec773max.csv'
fva_773max_outfile='./analysis/fva_773max.csv'
fva_max_diff_outfile='./analysis/fva_max_diff.csv'

fva_distrbutionmax_fig='./analysis/fva_distrbutionmax.png'

## ecGEM

In [6]:
ecGEM_rxn = pd.read_csv(ecGEM_rxn_file)
ecGEM_rxn.set_index('id', inplace=True)
enz_model=get_enzyme_constraint_model(json_model_file)
enz_model.reactions.get_by_id(substrate).bounds=(def_sub_cons,0)
enz_model.reactions.get_by_id(substrate+'_reverse').bounds=(0,0)
enz_model.reactions.get_by_id(obj).bounds=(0,umax) 
enz_model.objective=obj
ecFVA = flux_variability_analysis(enz_model, enz_model.reactions, loopless=False, fraction_of_optimum=0.9)
ecFVA.loc[ecFVA[(ecFVA['maximum']<=1E-6)].index,['maximum']]=0
ecFVA.loc[ecFVA[(ecFVA['minimum']<=1E-6)].index,['minimum']]=0
ecFVA.append(ecGEM_rxn)
ecFVA['ecMTMdifferernce'] = ecFVA['maximum'] - ecFVA['minimum']
#ecFVA = ecFVA.rename(columns={'maximum': 'ec_maximum', 'minimum': 'ec_minimum'})
ecFVA.reset_index()
ecFVA_name = pd.concat([ecGEM_rxn,ecFVA],axis = 1)
ecFVA_name = ecFVA_name.iloc[:, 4:7]

#isozyme
rex_name = []
for row in ecFVA_name.itertuples():
    if re.search('num', str(row.Index)):
        rex_name.append(str(row.Index.split('_num')[0]))
    else:
        rex_name.append(str(row.Index))
ecFVA_name['rex_name'] = rex_name

#FV max
FV_max = ecFVA_name.sort_values('ecMTMdifferernce', ascending=False).drop_duplicates(subset=['rex_name'], keep='first')
FV_max['id'] = FV_max.index
FV_max1 = FV_max.set_index('rex_name', drop=False)

#combine reverse reaction FV
reverse_list = []
rex_list = []
for row in FV_max1.itertuples():
    rex_list.append(str(row.Index))
    if re.search('reverse', str(row.Index)):
        reverse_list.append(str(row.Index).split('_reverse')[0])
FV_max_reverse = pd.DataFrame()
for row in FV_max1.itertuples():
    if str(row.Index) and str(row.Index)+'_reverse' in rex_list:
        FV_max_reverse.loc[row.Index,'id'] = row.Index
        FV_max_reverse.loc[row.Index,'ecMTMdifferernce'] = FV_max1.loc[row.Index, 'ecMTMdifferernce'] - FV_max1.loc[str(row.Index) + '_reverse', 'ecMTMdifferernce']
        FV_max_reverse.loc[row.Index,'ec_maximum'] = FV_max1.loc[row.Index, 'maximum']
        FV_max_reverse.loc[row.Index,'ec_minimum'] = FV_max1.loc[row.Index, 'minimum']
    elif re.search('reverse', str(row.Index)):
        FV_max_reverse.loc[str(row.Index).split('_reverse')[0],'id'] = str(row.Index).split('_reverse')[0]
        FV_max_reverse.loc[str(row.Index).split('_reverse')[0],'ecMTMdifferernce'] = FV_max1.loc[str(row.Index).split('_reverse')[0], 'ecMTMdifferernce'] - FV_max1.loc[row.Index, 'ecMTMdifferernce']
        FV_max_reverse.loc[str(row.Index).split('_reverse')[0],'ec_maximum'] = FV_max1.loc[row.Index, 'maximum']
        FV_max_reverse.loc[str(row.Index).split('_reverse')[0],'ec_minimum'] = FV_max1.loc[row.Index, 'minimum']
    else:
        FV_max_reverse.loc[row.Index,'id'] = row.Index
        FV_max_reverse.loc[row.Index,'ecMTMdifferernce']  = FV_max1.loc[row.Index, 'ecMTMdifferernce'] 
        FV_max_reverse.loc[row.Index,'ec_maximum'] = FV_max1.loc[row.Index, 'maximum'] 
        FV_max_reverse.loc[row.Index,'ec_minimum'] = FV_max1.loc[row.Index, 'minimum'] 
ecFV_result = FV_max_reverse.set_index('id', inplace=False)
ecFV_result.to_csv(fva_ec773max_outfile)

0.128


## GEM

In [7]:
GEM_rxn = pd.read_csv(GEM_rxn_file)
GEM_rxn.set_index('id', inplace=True)
cg_model=cobra.io.read_sbml_model(sbml_path)
cg_model.reactions.get_by_id(substrate).bounds=(def_sub_cons,0)
cg_model.reactions.get_by_id(obj).bounds=(0,umax)
cg_model.objective=obj
GEM_FVA = flux_variability_analysis(cg_model, cg_model.reactions, loopless=False)
GEM_FVA.loc[GEM_FVA[(GEM_FVA['maximum']<=1E-6)].index,['maximum']]=0
GEM_FVA.loc[GEM_FVA[(GEM_FVA['minimum']<=1E-6)].index,['minimum']]=0
GEM_FVA.append(GEM_rxn)
GEM_FVA['iYW1475differernce'] = GEM_FVA['maximum'] - GEM_FVA['minimum']
GEM_FVA.to_csv(fva_773max_outfile)
# combine results
FVA_total = pd.concat([GEM_FVA,ecFV_result],axis = 1)
for index, row in FVA_total.iterrows():
    try:
        FVA_total.loc[index,'kcat_MW'] = enz_model.reactions.get_by_id(index).kcat
    except:
        pass
    try:
        FVA_total.loc[index,'reaction'] = cg_model.reactions.get_by_id(index).reaction
    except:
        pass
FVA_total.to_csv(fva_max_diff_outfile)

In [8]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go

# 读取CSV文件到DataFrame
df_fv = pd.read_csv(fva_max_diff_outfile)

# 列名
col1 = 'ecMTMdifferernce'
col2 = 'iYW1475differernce'


# 选择列并进行数据转换和筛选
df_ec = df_fv[col1]
df_ec = abs(df_ec)
df_ec.replace(0, np.nan, inplace=True)
df_ec = df_ec[df_ec >= 1.0E-6]

df_GEM = df_fv[col2]
df_GEM = abs(df_GEM)
df_GEM.replace(0, np.nan, inplace=True)
df_GEM = df_GEM[df_GEM >= 1.0E-6]



# 排序和重置索引
df_ec.sort_values(axis=0, inplace=True)
df_ec = df_ec.reset_index(drop=True)
df_GEM.sort_values(axis=0, inplace=True)
df_GEM = df_GEM.reset_index(drop=True)

# 计算y轴数据
y_index1 = df_ec.index / (len(df_ec) - 1)
y_index2 = df_GEM.index / (len(df_GEM) - 1)
data1_cdf_data = df_ec
data2_cdf_data = df_GEM
trace0 = go.Scatter(x=data2_cdf_data, y=y_index2, line={'color': 'blue', 'width': 4}, mode='lines', name='iYW1475', xaxis='x2', yaxis="y2")
trace1 = go.Scatter(x=data1_cdf_data, y=y_index1, mode='lines', name='ecMTM', line={'color': 'red', 'width': 4})

# 创建布局
layout = go.Layout(
    plot_bgcolor='white',
    xaxis=dict(title=dict(text="<b>Flux variability range [mmol/gDWh]</b>", font=dict(size=20, family='Times New Roman')),
               type="log",
               rangemode="tozero",
               range=[-6.2, 3],
               tickfont=dict(color='black', size=15, family='Times New Roman'),
               linecolor='black',
               ticks='inside',
               tickcolor='black',
               showexponent='all',
               exponentformat="power",
               zeroline=False,
              ),
    xaxis2=dict(
        linecolor='black',
        tickcolor='black',
        showticklabels=False,
        range=[-6.2, 3],
        type="log",
        tickfont=dict(color='black', size=15, family='Times New Roman'),
        rangemode="tozero",
        overlaying='x',
        side='top',
        nticks=5500,
        zeroline=False,
        showexponent='all',
        exponentformat="power",
        gridcolor="white"
    ),
    yaxis=dict(title=dict(text="<b>Cumulative distribution</b>", font=dict(size=20, family='Times New Roman')),
               range=[0, 1],
               showgrid=False,
               zeroline=False,
               rangemode="tozero",
               tickfont=dict(color='black', size=15, family='Times New Roman'),
               ticks='inside',
               tickcolor='black',
               linecolor='black'),
    yaxis2=dict(
        range=[0, 1],
        linecolor='black',
        tickcolor='black',
        showgrid=False,
        zeroline=False,
        rangemode="tozero",
        tickfont=dict(color='black', size=15, family='Times New Roman'),
        showticklabels=False,
        overlaying='y',
        side='right'
    ),
    legend=dict(x=0.05, y=0.95, font=dict(size=15, color="black", family='Times New Roman')),
    height=450,
    width=800,
    margin=go.layout.Margin(l=0, r=20, b=0, t=10)
)

# 创建图表，将trace0和trace1全部包括在data列表中
fig = go.Figure(data=[trace0, trace1], layout=layout)
fig.add_hline(y=0.5, line_width=3, line_color="orange")

# 保存图表为图像文件
fig.write_image(fva_distrbutionmax_fig)

# 显示图表
fig.show()